<a href="https://colab.research.google.com/github/heroza/Keel/blob/main/haberman.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import myfunction as fu
import numpy as np
import pandas as pd
from typing_extensions import Counter
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn import svm
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split, StratifiedKFold
from imblearn.over_sampling import RandomOverSampler, SMOTE

In [30]:
df = pd.read_csv('haberman.dat', header=None)


In [31]:
df.head()

,0,1,2,3
0,38,59,2,negative
1,39,63,4,negative
2,49,62,1,negative
3,53,60,2,negative
4,47,68,4,negative


In [34]:
num_features=3

le = LabelEncoder()
X = df.loc[:, df.columns[0:num_features]].to_numpy()

scaler = MinMaxScaler()
X = scaler.fit_transform(X)

y = df.loc[:, df.columns[-1]].to_numpy()
y = le.fit_transform(y)
# y_train = y_train.reshape(-1)
print(X.shape)
print(Counter(y))

(306, 3)
Counter({0: 225, 1: 81})


In [39]:
skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
thresholds = [0.02, 0.05, 0.1, 0.3, 0.5]
ks = [3,5,7,9]
skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)

for k in ks:
  for threshold in thresholds:
    scores = []
    for i in range(3):
      for fold, (train_index, test_index) in enumerate(skf.split(X, y)):
        # print(f"Fold {fold+1}")
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        sia_model = fu.train_siamese_dense(X_train, y_train, X_test, y_test, num_classes=2, num_features=num_features, epochs_siamese=100, batch_size=16)

        # Initialize the SVM classifier
        clf = svm.SVC()

        X_train_resampled, y_train_resampled = fu.sia_smote_balance_dense(X_train, y_train, sia_model, threshold, maxdist_from_base=0.5, k = k)

        # Train the classifier on the training set
        clf.fit(X_train_resampled, y_train_resampled)

        # Predict the labels of the test set
        y_pred = clf.predict(X_test)

        # Calculate the F1 score
        score = fu.calculate_score(y_test, y_pred)
        scores.append(score)

        # Print the F1 score
        print(f"Score in i:{i}, fold:{fold}:", score)

    print(f'k={k} and threshold={threshold}')
    print('printing score')
    fu.print_score(scores)

creating pairs
Counter({1.0: 126, 0.0: 126})
Counter({1.0: 32, 0.0: 32})
init base network
training siamese network...
evaluate
2/2 [==============================] - 0s 6ms/step - loss: 0.2403
predict train
8/8 [==============================] - 0s 2ms/step
compute train acc
predict val
2/2 [==============================] - 0s 7ms/step
compute val acc
Loss = 0.24032174050807953, Train Accuracy = 0.7142857142857143 Test Accuracy = 0.59375
4/4 [==============================] - 0s 4ms/step
Score in i:0, fold:0: [0.5483870967741935, 0.615686274509804, 0.5973799001456604, 0.48148148148148157, 0.35135135135135137, 0.7647058823529411, 0.4666666666666667, 0.6156862745098038]
creating pairs
Counter({1.0: 128, 0.0: 128})
Counter({1.0: 30, 0.0: 30})
init base network
training siamese network...
evaluate
2/2 [==============================] - 0s 7ms/step - loss: 0.2711
predict train
8/8 [==============================] - 0s 2ms/step
compute train acc
predict val
2/2 [===========================

In [40]:
skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
thresholds = [0.02, 0.05, 0.1, 0.3, 0.5]
ks = [3,5,7,9]
skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)

for k in ks:
  for threshold in thresholds:
    scores = []
    for i in range(3):
      for fold, (train_index, test_index) in enumerate(skf.split(X, y)):
        # print(f"Fold {fold+1}")
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        sia_model = fu.train_siamese_dense(X_train, y_train, X_test, y_test, num_classes=2, num_features=num_features, epochs_siamese=100, batch_size=16)

        # Initialize the SVM classifier
        clf = svm.SVC()

        X_train_resampled, y_train_resampled = fu.sia_smote_then_smote_dense(X_train, y_train, sia_model, threshold, maxdist_from_base=0.5, k = k)

        # Train the classifier on the training set
        clf.fit(X_train_resampled, y_train_resampled)

        # Predict the labels of the test set
        y_pred = clf.predict(X_test)

        # Calculate the F1 score
        score = fu.calculate_score(y_test, y_pred)
        scores.append(score)

        # Print the F1 score
        print(f"Score in i:{i}, fold:{fold}:", score)

    print(f'k={k} and threshold={threshold}')
    print('printing score')
    fu.print_score(scores)

creating pairs
Counter({1.0: 126, 0.0: 126})
Counter({1.0: 32, 0.0: 32})
init base network
training siamese network...
evaluate
2/2 [==============================] - 0s 8ms/step - loss: 0.2484
predict train
8/8 [==============================] - 0s 2ms/step
compute train acc
predict val
2/2 [==============================] - 0s 5ms/step
compute val acc
Loss = 0.24843288958072662, Train Accuracy = 0.7023809523809523 Test Accuracy = 0.5
4/4 [==============================] - 0s 3ms/step
Score in i:0, fold:0: [0.5483870967741935, 0.5973856209150327, 0.587450456818008, 0.46153846153846156, 0.34285714285714286, 0.7058823529411765, 0.4888888888888889, 0.5973856209150328]
creating pairs
Counter({1.0: 128, 0.0: 128})
Counter({1.0: 30, 0.0: 30})
init base network
training siamese network...
evaluate
2/2 [==============================] - 0s 6ms/step - loss: 0.2795
predict train
8/8 [==============================] - 0s 4ms/step
compute train acc
predict val
2/2 [==============================]